#*Exercise:1*

#*Direct OLSLR*:
\begin{equation}
min_x f(x) = \frac{1}{2} \|Ax - y\|_2^2
\end{equation}
Let  $x^*_f =arg min_x f(x)$

#*Regularized OLSLR*:
\begin{equation}
min_x f_\lambda(x) = \frac{\lambda}{2}x^\top x + \frac{1}{2} \|Ax - y\|_2^2
\end{equation}
Let  $x^*_{f_\lambda} =argmin_x f_\lambda(x)$


#*Importing required packages*

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from timeit import default_timer as timer
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
from sklearn.datasets import load_digits
import warnings
warnings.filterwarnings("ignore")

#*Loading the digits dataset, creating the data matrix A and label vector y*

In [ ]:
digits = load_digits()
print(digits.data.shape)     ## Check the shape of digits data
print(digits.target.shape)   ## Check the shape of digits target
A = digits.data
#In the following code, we create a Nx1 vector of target labels
y = 1.0*np.ones([A.shape[0],1])
for i in range(digits.target.shape[0]):
    y[i] = digits.target[i]

(1797, 64)
(1797,)


 # *Method to compute and return the function value*

In [ ]:
def evalf(A, x, y):  
    #Input: x is a numpy array of size 64
    assert A.shape[0] == 1797 and A.shape[1] == 64
    assert type(x) is np.ndarray and x.shape[0] == A.shape[1] and x.shape[1] == 1 
    assert type(y) is np.ndarray and y.shape[0] == A.shape[0] and y.shape[1] == 1
    temp = np.subtract(np.dot(A,x),y)
    fval = 0.5*np.linalg.norm(temp,2)**2
    return fval


def evalf_reg(lambd, A, x, y):  
    #Input: x is a numpy array of size 64
    assert type(lambd) is float
    assert A.shape[0] == 1797 and A.shape[1] == 64
    assert type(x) is np.ndarray and x.shape[0] == A.shape[1] and x.shape[1] == 1 
    assert type(y) is np.ndarray and y.shape[0] == A.shape[0] and y.shape[1] == 1
    
    temp = np.subtract(np.dot(A,x),y)
    fval = np.multiply((lambd/2), np.dot(x.T,x)) + 0.5*np.linalg.norm(temp,2)**2
    return fval

#*Method to compute and return the gradient value*

In [ ]:
def evalg(A, x, y):  
    #Input: x is a numpy array of size 64
    assert A.shape[0] == 1797 and A.shape[1] == 64
    assert type(x) is np.ndarray and x.shape[0] == A.shape[1] and x.shape[1] == 1 
    assert type(y) is np.ndarray and y.shape[0] == A.shape[0] and y.shape[1] == 1
    temp = np.dot(A,x) - y
    grad = np.dot(A.T,temp)
    return grad

def evalg_reg(lambd, A, x, y):  
    #Input: x is a numpy array of size 64
    assert type(lambd) is float
    assert A.shape[0] == 1797 and A.shape[1] == 64
    assert type(x) is np.ndarray and x.shape[0] == A.shape[1] and x.shape[1] == 1 
    assert type(y) is np.ndarray and y.shape[0] == A.shape[0] and y.shape[1] == 1
    temp = np.dot(A,x) - y
    grad = np.add(np.multiply(lambd,x), np.dot(A.T,temp))
    return grad
    

#*Method to compute and return the Hessian value*

In [ ]:
def evalh(A, x):
    assert type(x) is np.ndarray and x.shape[0] == A.shape[1] and x.shape[1] == 1 
    assert A.shape[0] == 1797 and A.shape[1] == 64
    hess = np.matmul(A.T, A)
    return hess

def evalh_reg(lambd, A, x):
    assert type(lambd) is float
    assert type(x) is np.ndarray and x.shape[0] == A.shape[1] and x.shape[1] == 1 
    assert A.shape[0] == 1797 and A.shape[1] == 64
    hess = np.multiply(lambd,np.identity(A.shape[1])) + np.matmul(A.T, A)
    return hess

#*Method to construct D_k matrix used in Newton's method*

In [ ]:
def compute_D_k_newton(A, x):
    assert A.shape[0] == 1797 and A.shape[1] == 64
    assert type(x) is np.ndarray and x.shape[0] == A.shape[1] and x.shape[1] == 1 
    d = np.linalg.inv(evalh(A, x))
    return d

def compute_D_k_newton_reg(lambd, A, x):
    assert type(lambd) is float
    assert A.shape[0] == 1797 and A.shape[1] == 64
    assert type(x) is np.ndarray and x.shape[0] == A.shape[1] and x.shape[1] == 1 
    d = np.linalg.inv(evalh_reg(lambd, A, x))
    return d

#*Method to compute the step length using backtracking*

In [ ]:
def compute_steplength_backtracking_scaled_direction(A, x, y, gradf, D_k, alpha_start, rho, gamma): 
    assert A.shape[0] == 1797 and A.shape[1] == 64    
    assert type(x) is np.ndarray and x.shape[0] == A.shape[1] and x.shape[1] == 1  
    assert type(y) is np.ndarray and y.shape[0] == A.shape[0] and y.shape[1] == 1
    assert type(gradf) is np.ndarray and len(gradf) == 64 
    assert type(alpha_start) is float and alpha_start>=0. 
    assert type(rho) is float and rho>=0.
    assert type(gamma) is float and gamma>=0. 
    assert type(D_k) is np.ndarray and len(D_k) == 64   
    alpha = alpha_start
    p_k = -np.dot(D_k,gradf)
    while evalf(A, np.add(x,np.multiply(alpha,p_k)),y) > np.multiply(gamma*alpha,np.dot(gradf.T,p_k)) + evalf(A,x,y):
        alpha = rho*alpha
    return alpha

def compute_steplength_backtracking_scaled_direction_reg(lambd, A, x, y, gradf, D_k, alpha_start, rho, gamma): 
    assert type(lambd) is float    
    assert A.shape[0] == 1797 and A.shape[1] == 64    
    assert type(x) is np.ndarray and x.shape[0] == A.shape[1] and x.shape[1] == 1  
    assert type(y) is np.ndarray and y.shape[0] == A.shape[0] and y.shape[1] == 1
    assert type(gradf) is np.ndarray and len(gradf) == 64 
    assert type(alpha_start) is float and alpha_start>=0. 
    assert type(rho) is float and rho>=0.
    assert type(gamma) is float and gamma>=0. 
    assert type(D_k) is np.ndarray and len(D_k) == 64  
    alpha = alpha_start
    p_k = -np.dot(D_k,gradf)
    while evalf_reg(lambd, A, np.add(x,np.multiply(alpha,p_k)),y) > np.multiply(gamma*alpha,np.dot(gradf.T,p_k)) + evalf_reg(lambd, A,x,y):
        alpha = rho*alpha
    return alpha

#*Line search type*

In [ ]:
CONSTANT_STEP_LENGTH = 3
BACKTRACKING_LINE_SEARCH = 2
EXACT_LINE_SEARCH = 1

#*Method to find the minimizer using Newton's method*

In [ ]:
def find_minimizer_Newtonmethod(start_x, A, y, tol, line_search_type, *args):
    #Start_x is a numpy array of size n, tol denotes the tolerance and is a positive float value
    assert A.shape[0] == 1797 and A.shape[1] == 64      
    assert type(start_x) is np.ndarray and start_x.shape[0] == A.shape[1] and start_x.shape[1] == 1 
    assert type(y) is np.ndarray and y.shape[0] == A.shape[0] and y.shape[1] == 1    
    assert type(tol) is float and tol>=0 

    x = start_x
    g_x = evalg(A, x, y)

    x_array = [x]
    f_array = [evalf(A, x,y)]
    if line_search_type == BACKTRACKING_LINE_SEARCH:
        if args is None:
            err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
            raise ValueError(err_msg)
        elif len(args)<3 :
            err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
            raise ValueError(err_msg)
        else:
            alpha_start = float(args[0])
            rho = float(args[1])
            gamma = float(args[2])
    k = 0
    # print('iter:',k,  ' f(x):', evalf(x,n), ' gradient norm:', np.linalg.norm(g_x))
    while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
        d_k = compute_D_k_newton(A, x)
        if line_search_type == BACKTRACKING_LINE_SEARCH:
            step_length = compute_steplength_backtracking_scaled_direction(A, x, y, g_x, d_k, alpha_start, rho, gamma) 
        elif line_search_type == CONSTANT_STEP_LENGTH: 
            step_length = 1.0
        else:  
            raise ValueError('Line search type unknown. Please check!')
        
        # Gradient descent steps
        x = np.subtract(x, np.multiply(step_length,np.dot(d_k, g_x))) 
        k += 1 
        g_x = evalg(A, x, y) 
        x_array.append(x)
        f_array.append(evalf(A, x,y))
        # print('iter:',k, ' x:', x, ' f(x):', evalf(A, x, y), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
    return x_array, f_array, k

#*Method to find the minimizer using Newton's method for regularized OLSLR*

In [ ]:
def find_minimizer_Newtonmethod_reg(lambd, start_x, A, y, tol, line_search_type, *args):
    assert type(lambd) is float
    assert A.shape[0] == 1797 and A.shape[1] == 64      
    assert type(start_x) is np.ndarray and start_x.shape[0] == A.shape[1] and start_x.shape[1] == 1 
    assert type(y) is np.ndarray and y.shape[0] == A.shape[0] and y.shape[1] == 1    
    assert type(tol) is float and tol>=0 
    x = start_x
    g_x = evalg_reg(lambd, A, x, y)
    x_array = [x]
    f_array = [evalf_reg(lambd, A, x,y)]
    if line_search_type == BACKTRACKING_LINE_SEARCH:
        if args is None:
            err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
            raise ValueError(err_msg)
        elif len(args)<3 :
            err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
            raise ValueError(err_msg)
        else:
            alpha_start = float(args[0])
            rho = float(args[1])
            gamma = float(args[2])
    k = 0
    # print('iter:',k,  ' f(x):', evalf(x,n), ' gradient norm:', np.linalg.norm(g_x))
    while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
        d_k = compute_D_k_newton_reg(lambd, A, x)
        if line_search_type == BACKTRACKING_LINE_SEARCH:
            step_length = compute_steplength_backtracking_scaled_direction_reg(lambd, A, x, y, g_x, d_k, alpha_start, rho, gamma) 
        elif line_search_type == CONSTANT_STEP_LENGTH: 
            step_length = 1.0
        else:  
            raise ValueError('Line search type unknown. Please check!') 
        # Gradient descent steps
        x = np.subtract(x, np.multiply(step_length,np.dot(d_k, g_x))) 
        k += 1 
        g_x = evalg_reg(lambd, A, x, y) 
        x_array.append(x)
        f_array.append(evalf_reg(lambd, A, x,y))
        if k % 2000 == 0:
            print('iter:',k, ' f(x):', evalf_reg(lambd, A, x, y), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
            print('_______________________________________')
    return x_array, f_array, k

#*Method to find minimizer using BFGS method* 

In [ ]:
def find_minimizer_BFGS(start_x, A, y, tol, line_search_type, *args):
    assert A.shape[0] == 1797 and A.shape[1] == 64      
    assert type(start_x) is np.ndarray and start_x.shape[0] == A.shape[1] and start_x.shape[1] == 1 
    assert type(y) is np.ndarray and y.shape[0] == A.shape[0] and y.shape[1] == 1    
    assert type(tol) is float and tol>=0 

    x = start_x
    g_x = evalg(A, x, y)

    if line_search_type == BACKTRACKING_LINE_SEARCH:
        if args is None:
            err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
            raise ValueError(err_msg)
        elif len(args)<3 :
            err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
            raise ValueError(err_msg)
        else:
            alpha_start = float(args[0])
            rho = float(args[1])
            gamma = float(args[2])
    k = 0
    B = np.identity(64)
    x_array = [x]
    f_array = [evalf(A, x,y)]

    while (np.linalg.norm(g_x) > tol): 
        p = -np.dot(B, g_x)
        if line_search_type == BACKTRACKING_LINE_SEARCH:
            d_k = B
            step_length = compute_steplength_backtracking_scaled_direction(A, x, y, g_x, d_k, alpha_start, rho, gamma) 
        elif line_search_type == CONSTANT_STEP_LENGTH: 
            step_length = 1.0
        else:  
            raise ValueError('Line search type unknown. Please check!')
        
        # Gradient descent steps
        x_next = np.add(x, np.multiply(step_length,p)) 
        s_k = x_next - x
        y_k = evalg(A, x_next,y) - g_x
        mu_k = 1.0/np.dot(y_k.T, s_k)
        temp = (np.identity(64) - np.multiply(mu_k,np.outer(y_k, s_k.T)))
        B = np.dot(temp.T, np.dot(B, temp)) + np.multiply(mu_k, np.outer(s_k, s_k.T))
        k += 1 
        x = x_next
        g_x = evalg(A, x, y)
        
        x_array.append(x)
        f_array.append(evalf(A, x, y))
        # print('iter:',k, ' x:', x, ' f(x):', evalf(A, x, y), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

    return x_array, f_array, k

#*Method to find minimizer using BFGS method for regularized OLSLR*

In [ ]:
def find_minimizer_BFGS_reg(lambd, start_x, A, y, tol, line_search_type, *args):
    assert type(lambd) is float
    assert A.shape[0] == 1797 and A.shape[1] == 64      
    assert type(start_x) is np.ndarray and start_x.shape[0] == A.shape[1] and start_x.shape[1] == 1 
    assert type(y) is np.ndarray and y.shape[0] == A.shape[0] and y.shape[1] == 1    
    assert type(tol) is float and tol>=0 

    x = start_x
    g_x = evalg_reg(lambd, A, x, y)

    if line_search_type == BACKTRACKING_LINE_SEARCH:
        if args is None:
            err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
            raise ValueError(err_msg)
        elif len(args)<3 :
            err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
            raise ValueError(err_msg)
        else:
            alpha_start = float(args[0])
            rho = float(args[1])
            gamma = float(args[2])
    k = 0
    B = np.identity(64)
    x_array = [x]
    f_array = [evalf_reg(lambd, A, x,y)]

    while (np.linalg.norm(g_x) > tol): 
        p = -np.dot(B, g_x)
        if line_search_type == BACKTRACKING_LINE_SEARCH:
            d_k = B
            step_length = compute_steplength_backtracking_scaled_direction_reg(lambd, A, x, y, g_x, d_k, alpha_start, rho, gamma) 
        elif line_search_type == CONSTANT_STEP_LENGTH: 
            step_length = 1.0
        else:  
            raise ValueError('Line search type unknown. Please check!')
        
        # Gradient descent steps
        x_next = np.add(x, np.multiply(step_length,p)) 
        s_k = x_next - x
        y_k = evalg_reg(lambd, A, x_next,y) - g_x
        mu_k = 1.0/np.dot(y_k.T, s_k)
        temp = (np.identity(64) - np.multiply(mu_k,np.outer(y_k, s_k.T)))
        B = np.dot(temp.T, np.dot(B, temp)) + np.multiply(mu_k, np.outer(s_k, s_k.T))
        k += 1 
        x = x_next
        g_x = evalg_reg(lambd, A, x, y)
        
        x_array.append(x)
        f_array.append(evalf_reg(lambd, A, x, y))
        # print('iter:',k, ' x:', x, ' f(x):', evalf(A, x, y), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

    return x_array, f_array, k

#*Question 1:*


In [ ]:
my_start_x = np.zeros((64,1))
my_tol= 1e-5
alpha_start = 0.9
rho = 0.5
gamma = 0.5

x_array_newton, f_array_newton, iters_opt_newton = find_minimizer_Newtonmethod(my_start_x, A, y, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)


LinAlgError: ignored

In [ ]:
my_start_x = np.zeros((64,1))
my_tol= 1e-3
alpha_start = 0.9
rho = 0.5
gamma = 0.5
lambd = 0.001

x_array_newton_reg, f_array_newton_reg, iters_opt_newton_reg = find_minimizer_Newtonmethod_reg(lambd, my_start_x, A, y, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)


In [ ]:
print("Direct OLSLR using Newton:\n")
print("Minimizer:", x_array_newton_reg[-1].reshape(1,-1))
print("Min function: ",f_array_newton_reg[-1][0][0])
print("Number of iterations: ",iters_opt_newton_reg)


Direct OLSLR using Newton:

Minimizer: [[ 0.00000000e+00  9.69076881e-02 -4.32192761e-03 -7.75916338e-03
   7.49591986e-02  1.13946582e-02 -2.71293921e-02 -7.34410663e-03
   9.98267906e-01 -2.88089376e-02  1.18688356e-01  6.60922719e-02
  -5.57075914e-02 -6.97056149e-02  9.65844013e-02  2.55196013e-01
  -7.29830665e-01  2.42711745e-02  7.73241543e-02 -2.33008498e-02
  -5.64077618e-02  5.72413971e-02 -4.88675247e-02 -2.62555997e-01
  -9.06071168e-01 -1.49769638e-01  5.64022789e-02  8.96665318e-02
   8.39315937e-02  9.85410046e-02  1.69269848e-03 -2.96649687e+00
   0.00000000e+00 -1.54361470e-01 -9.32404639e-03  1.39497978e-01
  -3.69237437e-02  5.46111772e-02 -9.20425585e-03  0.00000000e+00
   1.03326506e-01  1.23983484e-01 -1.37635230e-02  5.40029021e-03
   1.31185699e-01  5.49577814e-02  2.24935899e-02  7.48046264e-03
   6.17507773e-01  2.44100619e-02  1.42356436e-03 -6.21114800e-02
  -2.07024630e-01 -3.38510517e-02  1.05486785e-01 -1.40336481e-01
  -9.82347826e-01 -1.14473639e-01  2.

#*Comment:*
> * In case of direct OLSLR using Newton method, we obtained Singular matrix error while computing the inverse of the Hessian due to which we are unable to obtain the optimal point.
> * In case of regularized OLSLR using Newton method, the required minimizer is obtained within 9 iterations, and the minimum function value is $3064.454188158828$ 


#*Question 2:*


In [ ]:
my_start_x = np.zeros((64,1))
my_tol= 1e-5
alpha_start = 0.9
rho = 0.5
gamma = 0.5
## For unregularized
x_array_bfgs, f_array_bfgs, iters_opt_bfgs = find_minimizer_BFGS(my_start_x, A, y, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)
## For regularized
x_array_bfgs_reg, f_array_bfgs_reg, iters_opt_bfgs_reg = find_minimizer_BFGS_reg(lambd, my_start_x, A, y, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)

In [ ]:
print("Direct OLSLR using BFGS:")
print("Minimizer:", x_array_bfgs[-1].reshape(1,-1))
print("Min function: ",f_array_bfgs[-1])
print("Number of iterations: ",iters_opt_bfgs)

print("Regularized OLSLR using BFGS:")
print("Minimizer:", x_array_bfgs_reg[-1].reshape(1,-1))
print("Min function: ",f_array_bfgs_reg[-1][0][0])
print("Number of iterations: ",iters_opt_bfgs_reg)

Direct OLSLR using BFGS:
Minimizer: [[ 0.00000000e+00  9.69033569e-02 -4.32277233e-03 -7.76028319e-03
   7.49594380e-02  1.13947198e-02 -2.71328245e-02 -7.33176334e-03
   9.98337968e-01 -2.88095538e-02  1.18688288e-01  6.60916265e-02
  -5.57069862e-02 -6.97063705e-02  9.65876439e-02  2.55182251e-01
  -7.29828608e-01  2.42709916e-02  7.73249597e-02 -2.33000278e-02
  -5.64086144e-02  5.72426822e-02 -4.88717684e-02 -2.62467763e-01
  -9.06562829e-01 -1.49767791e-01  5.64019538e-02  8.96663590e-02
   8.39318159e-02  9.85411936e-02  1.69317615e-03 -2.96805758e+00
   0.00000000e+00 -1.54362338e-01 -9.32361205e-03  1.39497628e-01
  -3.69234835e-02  5.46111776e-02 -9.20505069e-03  0.00000000e+00
   1.03279535e-01  1.23983258e-01 -1.37639605e-02  5.40087815e-03
   1.31185107e-01  5.49570758e-02  2.24938237e-02  7.47977912e-03
   6.17755029e-01  2.44122357e-02  1.42333037e-03 -6.21110760e-02
  -2.07025036e-01 -3.38506003e-02  1.05486736e-01 -1.40335957e-01
  -9.84168996e-01 -1.14467153e-01  2.104

#*Comment:*
>* In case of OLSLR using BFGS method,  the required minimizer obtained within 76 iterations, and the minimum function value is $3064.4477111757$
>* In case of regularized OLSLR using Newton method, the required minimizer obtained within 76 iterations, and the minimum function value is $3064.4541881588275$ 
>* The minimizer obtained in both methods, Newtons and BFGS are verymuch the same.
>* Newton method performs far better in the regularized setting, but was unable to converge in the unregularized setting.